In [1]:
import sys
from pprint import pprint
from time import sleep
sys.path.insert(0, "../")

from src import BusinessAnalystAI, BusinessAnalystAIConfig
from src.reasoner.weak import WeakBAReasonerConfig
from src.reasoner.weak.utils import WeakReasonerStatus, WeakReasonerState, WeakGlobalSignal, WeakStateSignal
from src.reasoner.utils import UserResponse

from src.reasoner.weak.states_handlers.us_communicator.utils import USClarifierState
from src.reasoner.weak.states_handlers.scenarios_communicator.utils import SClarifierState

from src.reasoner.weak.states_handlers.freq_communicator.utils import FRExtractorState
from src.reasoner.weak.states_handlers.frestr_communicator.utils import FRestrClarifierState

from src.reasoner.weak.states_handlers.sreq_communicator.utils import SReqClarifierState
from src.reasoner.weak.states_handlers.srestr_communicator.utils import SRestrClarifierState

/home/dzigen/miniconda3/envs/baai_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ba_ai = BusinessAnalystAI(config=BusinessAnalystAIConfig(reasoner_name='weak', reasoner_config=WeakBAReasonerConfig()))

In [3]:
ba_ai.reasoner.change_mode_to(new_mode='prod')

Необъодимо разработать цифрового наставника, который анализирует коммуникации менеджера с клиентами Банка, проверяет информацию на основе своей базы знаний (памяти) о продукте и даёт рекомендации. Наставник должен поддерживать обновление и изменение графа знаний, а также обеспечивать валидацию и верификацию ответов менеджеров согласно установленным правилам и критериям оценки.

В Банке разрабатываются системы анализа коммуникаций с клиентами. Среди таких систем создаются цифровые тренажеры для клиентских менеджеров, которые позволяют, отрабатывать навыки работы с клиентами, в том числе работу с возражениями, повышать квалификацию клиентских менеджеров, и в конечном итоге повышать конверсию продаж. Кроме того, системы анализа коммуникаций с клиентами позволяют развивать продукты Банка через обратную связь, включая вопросы, жалобы и предложения, таким образом обобщая клиентский опыт.  Для работы со слабоструктурированной информацией, содержащейся в коммуникациях, необходимы LLM агенты, которые должны понимать семантику сообщений и диалогов. Для того, чтобы такой агент мог накапливать знания, его нужно снабдить внешней памятью. Исследования, в том числе проведенные Сколтехом, показывают, что наиболее эффективным инструментом создания внешней памяти для LLM-агентов являются графы знаний.  Создание такого цифрового тренажера, использующего граф как внешнюю память LLM-агента, позволит значительно повысить качество работы менеджеров с клиентами, снизить отток и повысить лояльность клиентов, и таким образом позволит повысить прибыль Банка.

Программную библиотеку с следующим фцнкионалом: (1) валидация ответа менеджера в рамках текущей коммуникации с клиентом; (2) корректировка ответа менеджера в зависимости от сделанных ошибок; (3) рекомендация возможного ответа менеджера для клиента в рамках текущей коммуникации; (4) формирование пространства памяти цифрового наставника на основе графов знаний, которая будет хранить текущую историю диалога менеджера с клиентом, правила/манифест общения с клиентом и техническую/сопроводительную информацию об услугах банка, на которую должен обуславливаться менеджер (и цифровой наставник) при генерации ответов на вопросы клиента

Продуктом будут пользоваться клиентские менеджеры для проверки/валидации своих ответов и получения базовых вариантов ответа для клиентов на их вопросы 

Одним из наиболее перспективных SOTA подходов является идея создания полноценной памяти агентов, построенных на основе графов. В общих фреймворках для построения и отслеживания LLM агентов, таких как LangGraph [5], есть некоторая фунциональность для работы с памятью агентов. Однако мы будем опираться на более специализированные решения.  В текущем году было представлено несколько решений по построению внешней памяти агентов на графах, среди них можно выделить две работы: Mem0 ("mem-zero") [6] и A-Mem: Agentic Memory [7]. В качестве бенчмарка для тестирования перечисленных решений использовался датасет LOCOMO [8], содержащий диалоги и вопросы к ним. Средняя длина диалога составляет 26000 токенов, и несмотря на то, что в настоящее время LLMs имеют возможность обработки такой длины контекста, было показано, что эффективность использования информации из контекста при генерации падает при увеличении длины контекста [9]. Таким образом, бенчмарк LOCOMO дает возможность оценить эффективность сохранения и извлечения фактов из внешней памяти для использования LLM агентом при генерации сообщений в диалоге. Граф знаний в решении AriGraph [2, 3], на котором будет базироваться данная работа, сочетает в себе как структурированные элементы (узлы и связи в семантической памяти, связи между семантическими и эпизодическими узлами), так и неструктурированные (текст в эпизодических узлах), что представляет собой потенциальное преимущество перед вышеперечисленными подходами. В работе [2] были приведены SOTA метрики (Exact Match) подхода AriGraph на датасете HotpotQA. В ходе данного проекта предполагается улучшение алгоритма извлечения и фильтрации релевантных фактов из графа знаний в системе AriGraph. Подтверждение данных предположений на бенчмарке LOCOMO будет представлять решение на уровне мировой SOTA, и будет означать существенное развитие технологий построения LLM-агентов.

Никаких данных нет, но потебуется подготовить датасеты для оценки качества следующих функций: (1) валидация ответов клиентских менеджеров; (2) генерация рекомендаций / вариантов ответа на вопросы клиентов; (3) корректировка ответов менеджеров на вопросы клиентов

Интеграции с другими системами нет

Поскольку для надежной архитектуры памяти необходима интеграция как структурированных, так и неструктурированных данных, такая интеграция осуществляется параллельно с концепциями семантической и эпизодической памяти. Граф знаний представляет собой сеть взаимосвязанных семантических знаний, в то время как эпизодические воспоминания отображаются в виде эпизодических ребер, которые могут связывать множество отношений внутри графа.  Построение графа, а также валидация и корректировка ответа менеджера будут реализованы в виде агентов на основе LLM. Тестирование алгоритмов планируется проводить с использованием GigaChat (при наличии доступа к вычислительным и информационным ресурсам Сбербанка), либо с LLaMA и QWEN (на мощностях Сколтеха). Финальное решение будет реализовано с использованием GigaChat. Предлагаемый подход является развитием подхода AriGraph [2, 3], и будет осуществляться исследовательской командой, в состав которой будут входить авторы данной статьи из Сколтеха.

====

Мы знаем следующие метрики эффективности, за счёт которых можно выполнить оценку заявленных функций:  Faithfulness, Correctness, Relevancy, BLEU, METEOR, ROUGE. Какие метрики нам ещё стоит включить для оценки качества валидации, генерации и коррекции ответов клиентского менеджера?

У нас нет эталонных ответов. Такой датасет потребуется подготовить для оценки качества функциональности, указанной ранее.

Можно брать любые источники. Необходимо, чтобы заявленный функционал системы был переимспольем: возможна итеграция в другие предметные области и для других кейсов.

Потребуется отобрать базовые QA-датасеты. Далее на их основе синтезировать вопросно-ответные пары для оценки качества функционала по валидации, генерации, коррекции ответов клиентских  менеджеров

Мы пока не знаем какие датаскты будут использованы. Для оценки качества нам не важна специфика датасета

Мы пока не знаем. Какие варианты можете предложить?



In [4]:
ba_ai.reset()

In [5]:
while True:

    sleep(1)
    if ba_ai.reasoner.REASONER_STATUS == WeakReasonerStatus.waiting_interaction:
        if ba_ai.reasoner.REASONER_STATE == WeakReasonerState.funcreq_extracting and ba_ai.reasoner.MODULES_MAP[WeakReasonerState.funcreq_extracting].STATE.base_state == FRExtractorState.done:
            pprint(ba_ai.reqspec_model.specification_struct.reqs_info.fgroups)
            pprint(ba_ai.reqspec_model.specification_struct.reqs_info.functional)
        if ba_ai.reasoner.REASONER_STATE == WeakReasonerState.sysreq_crud and ba_ai.reasoner.MODULES_MAP[WeakReasonerState.sysreq_crud].STATE.base_state == SReqClarifierState.select_act:
            pprint(ba_ai.reqspec_model.specification_struct.system_info.requirements)
        output = ba_ai.reasoner.prepare_response()
        print(output)
        print(output.ba_text)
    elif ba_ai.reasoner.REASONER_STATUS == WeakReasonerStatus.waiting_response:
        if ba_ai.reasoner.REASONER_STATE == WeakReasonerState.userstories_crud and ba_ai.reasoner.MODULES_MAP[WeakReasonerState.userstories_crud].STATE.base_state == USClarifierState.select_act:
            pprint(ba_ai.reqspec_model.specification_struct.user_stories)
        elif ba_ai.reasoner.REASONER_STATE == WeakReasonerState.scenarios_ccrud and ba_ai.reasoner.MODULES_MAP[WeakReasonerState.scenarios_ccrud].STATE.base_state == SClarifierState.select_act:
            pprint(ba_ai.reqspec_model.specification_struct.scenarios)
        elif ba_ai.reasoner.REASONER_STATE == WeakReasonerState.funcrestr_ccrud and ba_ai.reasoner.MODULES_MAP[WeakReasonerState.funcrestr_ccrud].STATE.base_state == FRestrClarifierState.select_act:
            pprint(ba_ai.reqspec_model.specification_struct.reqs_info.functional_restrictions)
        elif ba_ai.reasoner.REASONER_STATE == WeakReasonerState.sysrestr_ccrud and ba_ai.reasoner.MODULES_MAP[WeakReasonerState.sysrestr_ccrud].STATE.base_state == SRestrClarifierState.select_act:
            pprint(ba_ai.reqspec_model.specification_struct.system_info.restrictions)

        user_text = input()
        user_response = UserResponse()
        if user_text == 'NEXT':
            user_response.signal = WeakGlobalSignal.next_stage
        elif user_text == 'ADD':
            user_response.signal = WeakStateSignal.add
        elif user_text == 'DELETE':
            user_response.signal = WeakStateSignal.delete
        elif user_text == 'EDIT':
            user_response.signal = WeakStateSignal.edit
        elif user_text == 'RECOMMEND':
            user_response.signal = WeakStateSignal.recommend_new
        elif user_text == 'APPROVE':
            user_response.signal = WeakStateSignal.approve
        elif user_text == 'DECLINE':
            user_response.signal = WeakStateSignal.decline
        elif user_text == 'CANCEL':
            user_response.signal = WeakStateSignal.cancel_operation
        else:
            user_response.user_text = user_text
        print(user_response)
        ba_ai.reasoner.process_response(user_response)
    elif ba_ai.reasoner.REASONER_STATE in [WeakReasonerState.done_unsupported_task, WeakReasonerState.done] and ba_ai.reasoner.REASONER_STATUS == WeakReasonerStatus.done:
        break
    else:
        raise ValueError

BAResponse(ba_text='Привет! Я ваш AI Бизнес Аналитик.', available_state_signals=[], available_global_signals=[], user_base_answer=None, is_userinput_locked=True)
Привет! Я ваш AI Бизнес Аналитик.
BAResponse(ba_text='Давайте начнём формировать техническое задание.', available_state_signals=[], available_global_signals=[], user_base_answer=None, is_userinput_locked=True)
Давайте начнём формировать техническое задание.
BAResponse(ba_text='Сейчас я задам вам несколько общих вопросов, чтобы лучше понять задачу.', available_state_signals=[], available_global_signals=[], user_base_answer=None, is_userinput_locked=True)
Сейчас я задам вам несколько общих вопросов, чтобы лучше понять задачу.
BAResponse(ba_text='Опишите вашу задачу простыми словами.', available_state_signals=[], available_global_signals=[], user_base_answer=None, is_userinput_locked=False)
Опишите вашу задачу простыми словами.
UserResponse(user_text='Необъодимо разработать цифрового наставника, который анализирует коммуникации м

In [6]:
print(ba_ai.reasoner.REASONER_STATE)
print(ba_ai.reasoner.REASONER_STATUS)

WeakReasonerState.done
WeakReasonerStatus.done


In [ ]:
pprint(ba_ai.reasoner.reqspec_model.specification_struct)

In [6]:
print(ba_ai.get_task_description())

# План проекта

## Назначение проект

Разработка программной библиотеки, представляющей собой цифрового наставника для клиентских менеджеров банка, способного анализировать коммуникации менеджеров с клиентами, проверять предоставленную менеджерами информацию на соответствие базе знаний банка и давать рекомендации по улучшению взаимодействия с клиентами. Система включает механизмы обновления и изменения графа знаний, обеспечивающего хранение технической и сопроводительной информации о продуктах банка, правил общения с клиентами и истории текущих диалогов. Основной целью разработки является повышение квалификации менеджеров, снижение оттока клиентов и увеличение лояльности путем улучшения качества обслуживания и повышения эффективности коммуникаций. Библиотека обеспечит функции валидации ответов менеджеров, корректировки этих ответов и формирования рекомендаций возможных ответов в процессе текущего диалога с клиентом.

## Основные задачи

* Разработать цифровую систему-наставника для под